In [0]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-414396f5-b177-f908-58bb-147eacbefddb)


In [0]:
!pip show keras-efficientnets

Name: keras-efficientnets
Version: 0.1.6.1
Summary: Keras implementation of EfficientNets of any configuration.
Home-page: https://github.com/titu1994/keras-efficientnets
Author: Somshubra Majumdar
Author-email: titu1994@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras, scipy, scikit-learn
Required-by: 


In [0]:
!pip install keras-efficientnets

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

 

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
#downloading the dataset
!gsutil cp gs://tibot-adnan/Food_dataset/food_dataset.zip food_dataset.zip

In [0]:
#unzipping the dataset
!unzip food_dataset.zip
!ls

In [0]:
#importing necessary packages
import numpy as np
import keras
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from math import ceil
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Activation
from keras.optimizers import SGD,adam,nadam
from keras.metrics import categorical_crossentropy
from keras import optimizers
from sklearn.metrics import confusion_matrix,roc_auc_score
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
%matplotlib inline

img_width, img_height = 224,224

train_data_dir = 'food_dataset/train'
test_data_dir=   'food_dataset/test'
nb_train_sample = 80800
nb_valid_sample = 10100
nb_test_sample=10100
#nb_test_sample=80
epochs = 50
batch_size = 60
multiplier=1
steps_train = ceil(multiplier*(nb_train_sample/batch_size))
steps_valid= ceil(multiplier*(nb_valid_sample/batch_size))
steps_test= ceil(nb_test_sample/1)
print("done")

done


In [0]:
'''Creating the training and testing image generator for feeding the images in the model
'''
train_datagen = ImageDataGenerator(
rescale=1. / 255,
channel_shift_range=30,
zoom_range=0.2,
width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
horizontal_flip=True,# randomly flip images

fill_mode='reflect',
validation_split= 0.1111111111111111)#for validation

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode='sparse',shuffle = True,
subset='training',seed=42) # set as training data

validation_generator = train_datagen.flow_from_directory(
train_data_dir,#same train directory
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode='sparse',shuffle = True,
subset='validation',seed=42) # set as validation data

# data generator for test set
test_datagen = ImageDataGenerator(rescale=1. / 255,)

# generator for reading test data from folder
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_width, img_height),
    
    color_mode = 'rgb',
    batch_size = 1,
    class_mode = 'sparse',
    shuffle = False)

In [0]:
#defining the model
from keras import backend as K
from keras_efficientnets import EfficientNetB1
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
 
base_model = EfficientNetB1( include_top=False, weights='imagenet') #defining the model according to the package description
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(101, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all  InceptionV3 layers
#for layer in model.layers[:-1]:
#    layer.trainable = False
model.summary()

In [0]:
#getting the model schema
a=model.to_json()
print(a)

**These following 3 cells are for determining the learning rate at the start of the training if needed. Run it if you wish to find the learning rate.**


In [0]:
from keras.callbacks import Callback
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

class LRFinder(Callback):
    def __init__(self, min_lr, max_lr, mom=0.9, stop_multiplier=None, 
                 reload_weights=True, batches_lr_update=5):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.mom = mom
        self.reload_weights = reload_weights
        self.batches_lr_update = batches_lr_update
        if stop_multiplier is None:
            self.stop_multiplier = -20*self.mom/3 + 10 # 4 if mom=0.9
                                                       # 10 if mom=0
        else:
            self.stop_multiplier = stop_multiplier
        
    def on_train_begin(self, logs={}):
        p = self.params
        try:
            n_iterations = p['epochs']*p['samples']//p['batch_size']
        except:
            n_iterations = p['steps']*p['epochs']
            
        self.learning_rates = np.geomspace(self.min_lr, self.max_lr, \
                                           num=n_iterations//self.batches_lr_update+1)
        self.losses=[]
        self.iteration=0
        self.best_loss=0
        if self.reload_weights:
            self.model.save_weights('tmp.hdf5')
        
    
    def on_batch_end(self, batch, logs={}):
        loss = logs.get('loss')
        
        if self.iteration!=0: # Make loss smoother using momentum
            loss = self.losses[-1]*self.mom+loss*(1-self.mom)
        
        if self.iteration==0 or loss < self.best_loss: 
                self.best_loss = loss
                
        if self.iteration%self.batches_lr_update==0: # Evaluate each lr over 5 epochs
            
            if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
          
            lr = self.learning_rates[self.iteration//self.batches_lr_update]            
            K.set_value(self.model.optimizer.lr, lr)

            self.losses.append(loss)            

        if loss > self.best_loss*self.stop_multiplier: # Stop criteria
            self.model.stop_training = True
                
        self.iteration += 1
    
    def on_train_end(self, logs=None):
        if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
                
        plt.figure(figsize=(12, 6))
        plt.plot(self.learning_rates[:len(self.losses)], self.losses)
        plt.xlabel("Learning Rate")
        plt.ylabel("Loss")
        plt.xscale('log')
        plt.show()

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
lr_finder = LRFinder(min_lr=0.00001, max_lr=1)

model.fit_generator(train_generator, steps_per_epoch=steps_train,callbacks=[lr_finder])

In [0]:
print(lr_finder.best_loss)
print(min(lr_finder.losses))
print(np.argmin(lr_finder.losses))
print(lr_finder.learning_rates[np.argmin(lr_finder.losses)])

RUN from Here

In [0]:
#Custom callback for getting learning rate each epoch
from keras.callbacks import Callback,History
from keras import backend as K
class showLR( Callback ) :
    def on_epoch_begin(self, epoch, logs=None):
        lr = float(K.get_value(self.model.optimizer.lr))
        print (" epoch={:02d}, lr={:.8f}".format( epoch, lr ))

In [0]:
#Defining optimizer and metrics for the model
import functools
top3_acc = functools.partial(keras.metrics.sparse_top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'
top5_acc = functools.partial(keras.metrics.sparse_top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'

showlr=showLR()
history=History()
checkpoint = ModelCheckpoint('FCEfnetB2-{epoch:03d}-04_08.h5', verbose=1, monitor='val_loss',save_best_only=False, mode='auto')
Adam = adam(lr=.0005, beta_1=0.9, beta_2=0.999999, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam,metrics=['accuracy',top3_acc,top5_acc])

In [0]:
model.fit_generator(train_generator,steps_per_epoch=steps_train,validation_data=validation_generator,
                    validation_steps=steps_valid,epochs=5,callbacks=[checkpoint,history,showlr], verbose=1)

In [0]:
#cell for testing the models
acc=[]
top3acc=[]
top5acc=[]
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

#for later use 
for i in range(1,6):
  file_number='{:03d}'.format(i)
  
  file_name='FCEfnetB2-'+file_number+'-04_08.h5'
  model.load_weights(file_name)
    test_generator.reset()
  predictions = model.predict_generator(test_generator,steps = steps_test,verbose=1)
  accuracy=accuracy_score(test_generator.classes,predictions.argmax(axis=1))*100
  print('top1 Accuracy for weight file {} = {:3f}%'.format(file_name,accuracy))
  top5 = 0.0  
  top3 = 0.0
  for idx, l in enumerate(test_generator.classes):
    prediction = predictions[idx]
    top3_values = (-prediction).argsort()[:3]
    if np.isin(np.array([l]), top3_values):
        top3 += 1.0
    top_values = (-prediction).argsort()[:5]
    if np.isin(np.array([l]), top_values):
        top5 += 1.0
        
    
  top3acc.append(top3/len(test_generator.classes)*100)
  top5acc.append(top5/len(test_generator.classes)*100)
  print("top3 acc= {:.03f}%".format(top3acc[i-1]))
  print("top5 acc= {:.03f}%".format(top5acc[i-1]))
  cm=confusion_matrix(test_generator.classes,predictions.argmax(axis=1))
  print(cm)
  print(classification_report(test_generator.classes, predictions.argmax(axis=1)))
  sns.heatmap(cm)
  plt.show()
  #roc_auc_score(test_generator.classes, predictions.argmax(axis=1))
  acc.append(accuracy)

In [0]:
print(acc)
print(top3acc)
print(top5acc)
print(np.max(acc))
print(np.max(top3acc))
print(np.max(top5acc))
print(np.argmax(acc))
print(np.argmax(top3acc))
print(np.argmax(top5acc))